In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [ ]:
#1

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import BooleanType

spark = SparkSession.builder.appName("Pattern Matching").getOrCreate()

data = [
("D/O New Delhi",),
("C\\O Tarun123 New Delhi",),
("C/D/ Kanisk  New Delhi $",)]

columns = ["Address"]
df = spark.createDataFrame(data, columns)

regex = "^[A-Za-z0-9 /\\\\]*New Delhi$"

import re

def matchPattern(add):
  if re.match(regex,str(add)):
    return True
  else:
    return False

matchPatternUDF = udf(matchPattern, BooleanType())

df = df.filter(matchPatternUDF(col("Address")))

df.show(truncate=False)

+----------------------+
|Address               |
+----------------------+
|D/O New Delhi         |
|C\O Tarun123 New Delhi|
+----------------------+



In [ ]:
#2.

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp, date_format
spark = SparkSession.builder.appName("timeconvert").getOrCreate()

data = [("2023-01-01 10:22",)]
columns = ["Time"]
df = spark.createDataFrame(data, columns)
df.show()

df = df.withColumn("Time", to_timestamp(col("Time"), "yyyy-MM-dd HH:mm"))
df.show()

df = df.withColumn("Time", date_format(col("Time"), "MM-yyyy-dd HH:mm:ss"))

df.show(truncate=False)


+----------------+
|            Time|
+----------------+
|2023-01-01 10:22|
+----------------+

+-------------------+
|               Time|
+-------------------+
|2023-01-01 10:22:00|
+-------------------+

+-------------------+
|Time               |
+-------------------+
|01-2023-01 10:22:00|
+-------------------+

